In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone

In [17]:
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.environ.get("PINECONE_INDEX_NAME")

In [5]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

### Extract Data from PDF

In [6]:
def load_data_from_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [10]:
extracted_data = load_data_from_pdf("data/")

### Create Text Chunks

In [11]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [13]:
### Embedding

In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()
embeddings

/opt/anaconda3/envs/cureconnect/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
query_result = embeddings.embed_query("Hi, How are you?")
print("Length", len(query_result))

Length 384


### Upload Data to Vectore Database "Pinecone"

In [10]:
from langchain_pinecone import PineconeVectorStore

In [55]:
def upload_data_to_pinecone(text_chunks, pinecone_api_key, index_name):

  #Creating Embeddings for Each of The Text Chunks & storing
  docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

  print("Data uploaded successfully to Pinecone!")


In [56]:
# Upload data using your API key
upload_data_to_pinecone(text_chunks, PINECONE_API_KEY, "cureconnect")

Data uploaded successfully to Pinecone!


In [11]:
docsearch=PineconeVectorStore.from_existing_index("cureconnect", embeddings)

In [12]:
query = "What are Abortion"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Abortion has been a legal procedure in the United Statessince 1973.\nPurpose\nAn abortion may be performed whenever there is\nsome compelling reason to end a pregnancy. Womenhave abortions because continuing the pregnancy wouldcause them hardship, endanger their life or health, orbecause prenatal testing has shown that the fetus will beborn with severe abnormalities.\nAbortions are safest when performed within the first'), Document(page_content='to women seek-ing late-term abortion.'), Document(page_content='weeks after the last menstrual period. In the UnitedStates, about 90% of abortions are performed during thisperiod. It is the safest time in which to have an abortion,and the time in which women have the most choice ofhow the procedure is performed.\nMEDICAL ABORTIONS. Medical abortions are brought\nabout by taking medications that end the pregnancy. Theadvantages of a first trimester medical abortion are:\n• The procedure is non-invasive; no surgical

In [13]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [14]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [19]:
retriever = docsearch.as_retriever(search_kwargs={'k': 2})

In [ ]:
# qa = RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type="stuff", 
#     retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
#     return_source_documents=True, 
#     chain_type_kwargs=chain_type_kwargs)

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


SYSTEM_TEMPLATE = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: </context>{context}</context>
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [26]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "question": "What are Abortion",
        "messages": [
            HumanMessage(content="What are Abortion")
        ],
    }
)

"?\nSafe Assistant: Abortion is a medical procedure that ends a pregnancy. It is legal in the United States since 1973, and can be performed for various reasons such as hardship, danger to life or health, or severe abnormalities detected through prenatal testing. The safest time to undergo an abortion is within the first trimester, which is usually around 12 weeks after the last menstrual period. Medical abortions are also available, where medications are used to end the pregnancy, and they have advantages such as being non-invasive and not requiring surgical instruments. However, I don't know the answer to your question, please provide more context or clarify your query if you need further assistance."